In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import ast

from multiprocessing import Pool

In [5]:
import tensorflow as tf
from tensorflow import keras


from tensorflow.keras.layers import Dense, Input, LSTM, GRU, Flatten, Reshape, Dropout, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint 
from tensorflow.keras.utils import plot_model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.losses import MeanSquaredError, CosineSimilarity

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Lectura y preprocesado de los datos

In [4]:
Y = np.load('../../Data/Y_10.npy')
X = np.load('../../Data/X_10.npy')

print('X shape:')
print(X.shape)

print('Y shape:')
print(Y.shape)

X shape:
(1000000, 10, 101)
Y shape:
(1000000, 101)


In [6]:
val_size = int(X.shape[0]*0.1) #10%

X_train = X[:-val_size,:,:]
X_val = X[-val_size:,:,:]

Y_train = Y[:-val_size,:]
Y_val = Y[-val_size:,:]

print('X train:')
print(X_train.shape)
print('Y train:')
print(Y_train.shape)

print('X val:')
print(X_val.shape)
print('Y val:')
print(Y_val.shape)

X train:
(900000, 10, 101)
Y train:
(900000, 101)
X val:
(100000, 10, 101)
Y val:
(100000, 101)


# BiLSTM

In [7]:
num_units = 128

In [8]:
input_shape = (X_train.shape[1],X_train.shape[2])

input_layer = Input(shape= input_shape, name='Inputs')

model_layers = Bidirectional(LSTM(units = num_units, return_sequences=True))(input_layer)
model_layers = Bidirectional(LSTM(units = num_units, return_sequences=True))(model_layers)
model_layers = Bidirectional(LSTM(units = num_units))(model_layers)


model_layers = Dense(Y_train.shape[1], activation='tanh', name='Output')(model_layers)

model_bilstm = Model(input_layer, model_layers, name ='model_bilstm')
model_bilstm.summary()

Model: "model_bilstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Inputs (InputLayer)          [(None, 10, 101)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 256)           235520    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10, 256)           394240    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               394240    
_________________________________________________________________
Output (Dense)               (None, 101)               25957     
Total params: 1,049,957
Trainable params: 1,049,957
Non-trainable params: 0
_________________________________________________________________


## Entrenamiento

In [9]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=2)

In [10]:
optimizer = Adam(learning_rate=1e-3)
loss_fn = MeanSquaredError()
model_bilstm.compile(optimizer= optimizer, loss = loss_fn)

In [ ]:
epochs = 100
batch_size = 32

history = model_bilstm.fit(X_train, Y_train, validation_data = (X_val, Y_val), epochs = epochs, batch_size=batch_size, callbacks=[callback], verbose=1)

Train on 900000 samples, validate on 100000 samples
Epoch 1/100
900000/900000 [==============================] - 178s 198us/sample - loss: 0.0554 - val_loss: 0.0545
Epoch 2/100
900000/900000 [==============================] - 174s 194us/sample - loss: 0.0544 - val_loss: 0.0541
Epoch 3/100
900000/900000 [==============================] - -17674s -19638us/sample - loss: 0.0539 - val_loss: 0.0540
Epoch 4/100
605824/900000 [===================>..........] - ETA: 1:24 - loss: 0.0536

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
plt.figure(figsize=(15,7))
plt.plot(history.history['loss'],label='Train loss')
plt.plot(history.history['val_loss'],label='Test loss')
plt.title('Loss')
plt.legend()
plt.grid()
plt.show()

In [17]:
model_bilstm.save('biLSTM_v1')

INFO:tensorflow:Assets written to: biLSTM_v1\assets


In [ ]:
predicted_latent_items = model_bilstm(np.float32(X[0:1,:,:])).numpy()

for i in tqdm(range(1, X.shape[0])):
    
    predicted_item = model_bilstm(np.float32(X[i:i+1,:,:])).numpy() 
    
    predicted_latent_items = np.vstack((predicted_latent_items,predicted_item))


  5%|███▉                                                                     | 54450/999999 [13:01<4:23:34, 59.79it/s]

In [ ]:
predicted_latent_items.shape

In [ ]:
np.save('predicted_biLSTM_latent_items', predicted_latent_items)